# DA210 Project 2 (data analysis)
## Tran Ong, Tieu Ngan, Anh Le
1. Data sources:
Attached link: 

  https://en.wikipedia.org/api/rest_v1/page/html/List_of_anime_franchises_by_episode_count

  https://en.wikipedia.org/wiki/List_of_highest-grossing_Japanese_films#Highest-grossing_Japanese_films_worldwide

2. The format of data with given sources were already in HTML format.

3. How did you access and parse the data (explained below).

4. Central questions:
What are the predictors for a high-grossing anime (only box office earnings), among rankings, number of views, ratings, franchise revenues, and anime formats? And are there relationships between any two factors among release time, ending time, runtime, episode count, OVA count, and total episode count? 


In [1]:
import datetime
import requests

In [2]:
import base64

In [3]:
from lxml import etree
import io

In [4]:
import numpy as np

At first step (after importing all libraries needed), we store the urls into variables and scrap the data with 'requests'.

In [5]:
animeUrl = 'https://en.wikipedia.org/api/rest_v1/page/html/List_of_anime_franchises_by_episode_count'
res = requests.get(animeUrl)

wiki_html = res.text
# print (wiki_html)

Next, we parse the data with 'io string' and get the root of the tree after parsing

In [6]:
htmlparser = etree.HTMLParser()
try:
    # This one should work
    tree = etree.parse(io.StringIO(wiki_html), parser=htmlparser)
    # root of the whole wiki page in HTML
    root = tree.getroot()
except:
    print("Failed to parse as HTML")

'performXML' function helps to convert XML to normal objects/primary data type in python (list,dictionary) and work with the converted data with pandas.

In [7]:
def performXML(node,pos):
  table = []
  index = []
  columns = []
  for x in node[pos].getchildren():
    for y in x.getchildren():
      # y: each row in the table 
      # print (etree.tostring(y, pretty_print=True).decode())
      row = []
      for c in y.getchildren():
        # c: each column/value of one row in the table 
        if c.tag == 'td':
          # print (etree.tostring(c, pretty_print=True).decode())
          if c.text != None:
            row.append(c.text.strip('\n'))
          else:
            a = c.find('i')
            if a != None:
              a = a.find('a').text
              row.append(a)
            else:
              row.append(0)
              # print (etree.tostring(c, pretty_print=True).decode())
        else:
          if c.text != None:
            columns.append(c.text)
          else:
            a = c.find('i')
            if a != None: 
              a = a.find('a').text
              # print (a)
              row.append(a)
      if row != []: table.append(row)
  return table, columns, index

In [8]:
node = root.xpath("//table[contains(@class,'wikitable')]")
# use of xpath to extract the node we will use in the future exactly/
table, columns, index = performXML(node,0)
# 0 is the index of the first and only table of the given path. 

In [9]:
# columns.pop(0)
print (columns)
print (table)

['No.', 'Title', 'Started Broadcasting', 'Finished Broadcasting', 'Runtime (TV)', 'Episode(s)(TV)', 'Other(s)', 'Total count']
[['1', 'Doraemon', '1 Apr 1973', 0, '22-24 minutes', '3,003', '67', '3,070'], ['2', 'Sazae-san', '5 Oct 1969', 0, '22-24 minutes', '2,640', 0, '2,640+'], ['3', 'Nintama Rantarō', '10 Apr 1993', 0, '10 minutes', '2,199', '3', '2,202'], ['4', 'Ojarumaru', '5 Oct 1998', 0, '10 minutes', 0, '4', '1,831'], ['5', 'Oyako Club', '3 Oct 1994', '30 Mar 2013', '5 minutes', '1,818', 0, '1,818'], ['6', 'Soreike! Anpanman', '3 Oct 1988', 0, '24 minutes', 0, '113', '1,700'], ['7', 'Kirin Monoshiri Yakata', '1 Jan 1975', '31 Dec 1979', '5 minutes', '1,565', 0, '1,565'], ['8', 'Chibi Maruko-chan', '7 Jan 1990', 0, '24 minutes', '1,493', '64', '1,557'], ['9', 'Kirin Ashita no Calendar', '1 Jan 1980', '30 Dec 1984', '5 minutes', '1,498', 0, '1,498'], ['10', 'Manga Nippon Mukashi Banashi', '7 Jan 1975', '2 Jan 1995', '24 minutes', '1,494', 0, '1,494'], ['11', 'Hoka Hoka Kazoku', '

In [10]:
for row in table:
  pop = row.pop(0)
  index.append(pop)
  if row[2] == 0: row[2] = 'Ongoing'
print (table)

[['Doraemon', '1 Apr 1973', 'Ongoing', '22-24 minutes', '3,003', '67', '3,070'], ['Sazae-san', '5 Oct 1969', 'Ongoing', '22-24 minutes', '2,640', 0, '2,640+'], ['Nintama Rantarō', '10 Apr 1993', 'Ongoing', '10 minutes', '2,199', '3', '2,202'], ['Ojarumaru', '5 Oct 1998', 'Ongoing', '10 minutes', 0, '4', '1,831'], ['Oyako Club', '3 Oct 1994', '30 Mar 2013', '5 minutes', '1,818', 0, '1,818'], ['Soreike! Anpanman', '3 Oct 1988', 'Ongoing', '24 minutes', 0, '113', '1,700'], ['Kirin Monoshiri Yakata', '1 Jan 1975', '31 Dec 1979', '5 minutes', '1,565', 0, '1,565'], ['Chibi Maruko-chan', '7 Jan 1990', 'Ongoing', '24 minutes', '1,493', '64', '1,557'], ['Kirin Ashita no Calendar', '1 Jan 1980', '30 Dec 1984', '5 minutes', '1,498', 0, '1,498'], ['Manga Nippon Mukashi Banashi', '7 Jan 1975', '2 Jan 1995', '24 minutes', '1,494', 0, '1,494'], ['Hoka Hoka Kazoku', '1 Oct 1976', '31 Mar 1982', '5 minutes', '1,428', 0, '1,428'], ['Shimajirō', '13 Dec 1993', 'Ongoing', '24 minutes', '1,403', '8', '1,41

In [11]:
runtime_convert = {'22-24 minutes': 22, '24 minutes': 24, '5 minutes': 5, '10 minutes': 10, '12 minutes': 12, '12-24 minutes': 12}
for x in table:
  x[5] = int(x[5])
  if type(x[4]) is str and len(x[4]) == 5: x[4] = int(x[4][0:1] + x[4][2:])
  else: 
    if x[4] == '62+148': x[4] = 62 + 148
    else: x[4] = int(x[4])
  if x[0] == 'Ojarumaru': x[4] = 1827
  if x[0] == 'Soreike! Anpanman': x[4] = 1587
  if x[0] == 'Pokémon': x[4] = 1220
  x[1] = int(x[1][-4:])
  if x[2] != 'Ongoing': x[2] = int(x[2][-4:])
  else: x[2] = 2022
  if type(x[3]) is str: x[3] = int(runtime_convert[x[3]])



From the above url, we create a dataframe for *the list of anime series by franchise series total episode count*. We only take a sample of 50 series. This data will also be created as a SQlite table called top_series. 


In [12]:
import pandas as pd

# 50 Anime series by franchise series total episode count
df = pd.DataFrame(table)
df.columns = ['Series name', 'Year release', 'End Broadcasting Year', 'Runtime (min on TV)', 'Episode count', 'Others', 'Total count']

In [13]:
df['Runtime (min on TV)'].replace(0, np.NaN, inplace = True)

In [14]:
df

,Series name,Year release,End Broadcasting Year,Runtime (min on TV),Episode count,Others,Total count
0,Doraemon,1973,2022,22.0,3003,67,"3,070"
1,Sazae-san,1969,2022,22.0,2640,0,"2,640+"
2,Nintama Rantarō,1993,2022,10.0,2199,3,"2,202"
3,Ojarumaru,1998,2022,10.0,1827,4,"1,831"
4,Oyako Club,1994,2013,5.0,1818,0,"1,818"
...,...,...,...,...,...,...,...
83,0,2002,2006,NaN,0,0,209
84,Katekyō Hitman Reborn!,2006,2010,NaN,0,0,204
85,The Kindaichi Case Files,1997,2016,NaN,0,0,200
86,Holly the Ghost,1991,1993,NaN,0,0,200


In [15]:
def convert2int(s):
  # print (type(s))
  if type(s) is str:
    s = s.replace(',', '')
    s = s.replace('+', '')
    return int(s)
  return s
total = df['Total count']
total_count = [convert2int(x) for x in total]
table_v1 = {'Series name': list(df['Series name']),'Year release': list(df['Year release']), 'End Broadcasting Year': list(df['End Broadcasting Year']), 'Runtime (min on TV)': list(df['Runtime (min on TV)']), 'Others': list(df['Others']), 'Total count': total_count}
df_v1 = pd.DataFrame(table_v1)
df_v1

,Series name,Year release,End Broadcasting Year,Runtime (min on TV),Others,Total count
0,Doraemon,1973,2022,22.0,67,3070
1,Sazae-san,1969,2022,22.0,0,2640
2,Nintama Rantarō,1993,2022,10.0,3,2202
3,Ojarumaru,1998,2022,10.0,4,1831
4,Oyako Club,1994,2013,5.0,0,1818
...,...,...,...,...,...,...
83,0,2002,2006,NaN,0,209
84,Katekyō Hitman Reborn!,2006,2010,NaN,0,204
85,The Kindaichi Case Files,1997,2016,NaN,0,200
86,Holly the Ghost,1991,1993,NaN,0,200


For DataFrame df - 50 Anime series by franchise series total episode count, we can find the relationship between any two of the following for anime series: Year release, End Broadcasting Time, Runtime (min on TV), Episode count, Others, Total count. 

In [16]:
# Grossing of Japanese Films
path = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_Japanese_films#Highest-grossing_Japanese_films_worldwide'

In [17]:
res = requests.get(path)

wiki_html = res.text
htmlparser = etree.HTMLParser()
try:
    # This one should work
    tree = etree.parse(io.StringIO(wiki_html), parser=htmlparser)
    # root of the whole wiki page in HTML
    root0 = tree.getroot()
except:
    print("Failed to parse as HTML")
node0 = root0.xpath("//table[contains(@class,'plainrowheaders')]")
# node0 = root0.xpath("//table[position()=1]")

Based on the data from the url, we create three DataFrames that reflect 3 lists of top Japanese anime movie: 
* Top 50 highest gross Japanese films worldwide - df0
* Top 50 highest gross japanese films in Japan - df1
* Top 50 box ticket sales all the time - df2

These lists help us answer the main question: What are the predictors for a high-grossing anime (only box office earnings), among rankings, number of views, ratings, franchise revenues, and anime formats?

In [18]:
#Top 50 highest gross Japanese films worldwide
table0, columns0, index0 = performXML(node0,0)
table0[0][3] = 'Anime'
table0[3][0] = "Howl's Moving Castle"
for x in table0:
  a = x[1][1:]
  a = a.replace(',', '')
  a = a.replace('$', '')
  x[1] = int(a)
print (table0)

[['Demon Slayer the Movie: Mugen Train', 506523013, '2020', 'Anime', 0], ['Spirited Away', 395580000, '2001', 'Anime', 0], ['Your Name', 380140500, '2016', 'Anime', 0], ["Howl's Moving Castle", 236323601, '2004', 'Anime', 0], ['One Piece Film: Red', 206740295, '2022', 'Anime', 0], ['Ponyo', 204826668, '2008', 'Anime', 0], ['Jujutsu Kaisen 0', 195870885, '2021', 'Anime', 0], ['Weathering with You', 193715360, '2019', 'Anime', 0], ['Stand by Me Doraemon', 183442714, '2014', 'Anime', 0], ['Pokémon: The First Movie', 172744662, '1998', 'Anime', 0], ['Princess Mononoke', 170005875, '1997', 'Anime', 0], ['Bayside Shakedown 2', 164450000, '2003', 0, 0], ['The Secret World of Arrietty', 149411550, '2010', 'Anime', 0], ['The Wind Rises', 136533257, '2013', 'Anime', 0], ['Pokémon: The Movie 2000', 133949270, '1999', 'Anime', 0], ['Dragon Ball Super: Broly', 122747755, '2018', 'Anime', 0], ['Detective Conan: The Fist of Blue Sapphire', 115570314, '2019', 'Anime', 0], ['Bayside Shakedown: The Movi

In [19]:
df0 = pd.DataFrame(table0)
df0.columns = ['Movie name', 'Revenue (USD)', 'Year release', 'Genre', 'a']
df0

,Movie name,Revenue (USD),Year release,Genre,a
0,Demon Slayer the Movie: Mugen Train,506523013,2020,Anime,0
1,Spirited Away,395580000,2001,Anime,0
2,Your Name,380140500,2016,Anime,0
3,Howl's Moving Castle,236323601,2004,Anime,0
4,One Piece Film: Red,206740295,2022,Anime,0
5,Ponyo,204826668,2008,Anime,0
6,Jujutsu Kaisen 0,195870885,2021,Anime,0
7,Weathering with You,193715360,2019,Anime,0
8,Stand by Me Doraemon,183442714,2014,Anime,0
9,Pokémon: The First Movie,172744662,1998,Anime,0


For dataframe df0 - "Top 50 highest gross Japanese films worldwide", we have three predictors : the year released of the movie, the movie revenue and the movie genre. These data will be used to compare Japanese films' revenue, year release, and genres in the location of Japan.

In [20]:
# Top 50 highest gross japanese films in Japan
table1, columns1, index1 = performXML(node0,1)
table1[0][1] = '¥40,430,000,000'
table1[0][3] = 'Anime'
table1[6][3] = 'Live-action'
for x in table1:
  a = x[1][1:]
  x[1] = int(a.replace(',', ''))
  
df1 = pd.DataFrame(table1)
print (len(table1))
# df1.rename(columns={df.columns[0]:'Movie name',df.columns[1]:'Revenue',df.columns[2]:'Year release',df.columns[3]:'Genre'},inplace=True)
df1.columns = ['Movie name', 'Revenue (Yen)', 'Year release', 'Genre', 'Other']
df1

26


,Movie name,Revenue (Yen),Year release,Genre,Other
0,Demon Slayer the Movie: Mugen Train,40430000000,2020,Anime,0
1,Spirited Away,31680000000,2001,Anime,0
2,Your Name,25030000000,2016,Anime,0
3,Princess Mononoke,20180000000,1997,Anime,0
4,Howl's Moving Castle,19600000000,2004,Anime,0
5,One Piece Film: Red,18670000000,2022,Anime,0
6,Bayside Shakedown 2,17350000000,2003,Live-action,0
7,Ponyo,15500000000,2008,Anime,0
8,Weathering with You,14190000000,2019,Anime,0
9,Jujutsu Kaisen 0,13750000000,2021,Anime,0


For dataFrame df1 - "Top 50 highest gross Japanese films in Japan", our predictors still keep as the same: the year released of the movie, the movie revenue and the movie genre. From these two DataFrames, we can analyze what kind of Japanese movie will be favorite both in Japan and worldwide, and from there predict what factor made to that success.

In [21]:
table2, columns2, index2 = performXML(node0,4)

In [22]:
print(table2)

[['Kimi yo Fundo no Kawa o Watare', '1976', '720,000', '433,700,000', '434,400,000', 'Live-action'], ['1978', '2,400,000', '100,000,000', '102,400,000', 'Live-action'], ['Sandakan No. 8', '1974', 'Un\xadknown', '100,000,000', '100,000,000', 'Live-action'], ['Legend of Dinosaurs & Monster Birds', '1977', '650,000', '48,700,000', '49,400,000', 'Live-action'], ['Spirited Away', '2001', '24,280,000', '22,095,221', '46,375,221', 'Anime'], ['Your Name', '2016', '19,300,000', '25,878,383', '45,178,383', 'Anime'], ['Demon Slayer the Movie: Mugen Train', '2020', '29,151,041', '15,354,900', '44,505,941', 'Anime'], ['Who Are You, Mr. Sorge?', '1961', 'Un\xadknown', '39,200,000', '39,200,000', 'Live-action'], ['Pokémon: The First Movie', '1998', '6,650,000', '30,187,706', '36,837,706', 'Anime'], ['Koi no Kisetsu', '1969', 'Un\xadknown', '27,600,000', '27,600,000', 'Live-action'], ['The Bullet Train', '1975', 'Un\xadknown', '25,440,638', '25,440,638', 'Live-action'], ['Tokyo Olympiad', '1965', '23,

In [23]:
table2[1] = ['Kitakitsune Monogatari'] + table2[1]
for x in table2:
  a,b,c = x[2],x[3],x[4]
  if a != 'Un\xadknown': a = int(a.replace(',',''))
  if b!= 'Un\xadknown' and b != '—': b = int(b.replace(',', ''))
  if c != 'Un\xadknown': c = int(c.replace(',', ''))
  x[2],x[3],x[4] = a,b,c
print (table2)

[['Kimi yo Fundo no Kawa o Watare', '1976', 720000, 433700000, 434400000, 'Live-action'], ['Kitakitsune Monogatari', '1978', 2400000, 100000000, 102400000, 'Live-action'], ['Sandakan No. 8', '1974', 'Un\xadknown', 100000000, 100000000, 'Live-action'], ['Legend of Dinosaurs & Monster Birds', '1977', 650000, 48700000, 49400000, 'Live-action'], ['Spirited Away', '2001', 24280000, 22095221, 46375221, 'Anime'], ['Your Name', '2016', 19300000, 25878383, 45178383, 'Anime'], ['Demon Slayer the Movie: Mugen Train', '2020', 29151041, 15354900, 44505941, 'Anime'], ['Who Are You, Mr. Sorge?', '1961', 'Un\xadknown', 39200000, 39200000, 'Live-action'], ['Pokémon: The First Movie', '1998', 6650000, 30187706, 36837706, 'Anime'], ['Koi no Kisetsu', '1969', 'Un\xadknown', 27600000, 27600000, 'Live-action'], ['The Bullet Train', '1975', 'Un\xadknown', 25440638, 25440638, 'Live-action'], ['Tokyo Olympiad', '1965', 23500000, 993555, 24493555, 'Live-action'], ['The Ballad of Narayama', '1983', 1600000, 2194

In [24]:
# Top 50 box ticket sales all the time
df2 = pd.DataFrame(table2)
# df2.rename(columns={df.columns[0]:'Year released',df.columns[1]:'Ticket sale in Japan',df.columns[2]:'Ticket sale oversea',df.columns[3]:'Ticket sale worldwide', df.columns[4]:'Movie genre'},inplace=True)
df2

,0,1,2,3,4,5
0,Kimi yo Fundo no Kawa o Watare,1976,720000,433700000,434400000,Live-action
1,Kitakitsune Monogatari,1978,2400000,100000000,102400000,Live-action
2,Sandakan No. 8,1974,Un­known,100000000,100000000,Live-action
3,Legend of Dinosaurs & Monster Birds,1977,650000,48700000,49400000,Live-action
4,Spirited Away,2001,24280000,22095221,46375221,Anime
5,Your Name,2016,19300000,25878383,45178383,Anime
6,Demon Slayer the Movie: Mugen Train,2020,29151041,15354900,44505941,Anime
7,"Who Are You, Mr. Sorge?",1961,Un­known,39200000,39200000,Live-action
8,Pokémon: The First Movie,1998,6650000,30187706,36837706,Anime
9,Koi no Kisetsu,1969,Un­known,27600000,27600000,Live-action


In [25]:
table4 = []
for x in table2:
  for y in table0:
    if x[0] == y[0]:
      table4.append(x + [y[1]])
      break

print (table4)

[['Spirited Away', '2001', 24280000, 22095221, 46375221, 'Anime', 395580000], ['Your Name', '2016', 19300000, 25878383, 45178383, 'Anime', 380140500], ['Demon Slayer the Movie: Mugen Train', '2020', 29151041, 15354900, 44505941, 'Anime', 506523013], ['Pokémon: The First Movie', '1998', 6650000, 30187706, 36837706, 'Anime', 172744662], ['Stand by Me Doraemon', '2014', 6250000, 16833251, 23083251, 'Anime', 183442714], ['Weathering with You', '2019', 10539869, 12147184, 22687053, 'Anime', 193715360], ["Howl's Moving Castle", '2004', 15500000, 6443336, 21943336, 'Anime', 236323601], ['Pokémon: The Movie 2000', '1999', 5600000, 13968660, 19568660, 'Anime', 133949270], ['Ponyo', '2008', 12900000, 5202354, 18102354, 'Anime', 204826668], ['Dragon Ball Super: Broly', '2018', 3070000, 14417445, 17487445, 'Anime', 122747755], ['Princess Mononoke', '1997', 14970000, 2014830, 16984830, 'Anime', 170005875], ['Detective Conan: The Fist of Blue Sapphire', '2019', 7400000, 7659418, 15059418, 'Anime', 1

At DataFrame df2 - box ticket sales all the time, we use a different factor: the ticket sale to comparing what is the gap among ticket sales in Japan, oversea and in worldwide. This step can help us to see the trending movie through the time, and analyze what factors that make a movie stays legend. 

In [26]:
ref_url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_Japanese_films#cite_ref-153'
res = requests.get(ref_url)

wiki_html = res.text

htmlparser = etree.HTMLParser()
try:
    # This one should work
    tree = etree.parse(io.StringIO(wiki_html), parser=htmlparser)
    # root of the whole wiki page in HTML
    root = tree.getroot()
except:
    print("Failed to parse as HTML")

node = root.xpath("//ol[contains(@class,'references')]")

table3 = []
for x in node[0].getchildren():
  a = []
  for y in x.getchildren():
    for z in y.getchildren():
      if z.tag == 'i':
        # print (z.text)
        a.append(z.text)
      if z.tag == 'ul':
        for t in z.getchildren():
          if t.tag == 'li':
            a.append(t.text)
  if len(a) > 0: table3.append(a)
  # print (etree.tostring(z, pretty_print = True).decode())

print (table3)

[['Kimi yo Fundo no Kawa o Watare', 'Manhunt', None, None], ['Mothra vs. Godzilla'], ['Spirited Away', 'China – 15,722,581', 'Europe – 2,995,684', 'United States and Canada – 1,961,800', 'South Korea (', 'Brazil and Argentina – 477,697'], ['Your Name', 'China – 20,728,000', 'South Korea – 3,712,891', 'Europe (excluding Russia and Spain) – 627,384', 'United States and Canada – 580,000', 'Russia and Argentina – 169,600', 'Spain – 60,508'], ['Demon Slayer the Movie: Mugen Train', 'Taiwan, Hong Kong, Macau, Southeast Asia, Latin America, Australia, New Zealand, Middle East, Africa – 6,340,000', 'United States and Canada – 5,090,000', 'South Korea – 2,151,861', 'Europe (excluding Russia) – 1,259,895', 'Russia – 513,144'], ['Pokémon: The First Movie', 'United States and Canada – 16,858,300', 'Europe – 12,679,046', 'Brazil – 468,000', 'South Korea (Seoul City) – 182,360'], ['The Bullet Train', 'France – 440,638', 'Soviet Union – 25,000,000+'], ['The Ballad of Narayama', None, 'France – 844,07

## SQL Database 

(requirement 6)
* Our first created table is top_series, implementing the top highest franchise anime series. We select the following columns to create the table: series_id, title, date_release, status, runtime_per_episode, episode_count, others. 

* Our second created table is top_movies, the top highest grossing anime movies. We select the following columns to create the table: movie_id, title, revenue, year_release, genre, others. Then we update a new column, or variable, called revenue_category. This new variable has the following values: 500M+, Between 100M and 500M, Between 50M and 100M. 

* At first, we need to import sqlite3 library to use it later

In [27]:
import sqlite3 as sql

* We also created a database called 'jp_films.db' to store the tables we will create in the next step.

In [28]:
conn = sql.connect('jp_films.db')  
cur = conn.cursor() 

In [29]:
# engine.execute("SELECT * FROM top_series").fetchall()

In [30]:
series_table = [x[:-1] for x in table]
movies_japan_revenue_table = table0

In [31]:
# films = sql.connect('films')
# film_cur = films.cursor()

* The two below queries help to create the two tables called 'top_series' and 'top_movies', respectively, with 'series_id' and 'movies_id', the two unique values in each table.

In [32]:
query = """
        CREATE TABLE IF NOT EXISTS top_series 
        ([series_id] INTEGER PRIMARY KEY, [title] TEXT, [date_release] TEXT, [status] TEXT, [runtime_per_episode] TEXT, [episode_count] INTEGER, [others] INTEGER)
        """
cur.execute(query)

In [33]:
query1 = """
        CREATE TABLE IF NOT EXISTS top_movies
        ([movie_id] INTEGER PRIMARY KEY, [title] TEXT, [revenue] INTEGER, [year_release] TEXT, [genre] TEXT, [others] INTEGER)
        """
cur.execute(query1)

* We also implement a function called 'insertion' to modify the empty tables that we created before. By inserting new values, we will have a complete/normal SQL databases.

In [34]:
def insertion(table, sqltable, cols, cursor):
  i = 1
  n = len(cols)
  for row in sqltable[0:50]:
    r = [i] + row
    b = str(tuple(r))
    # print (b)
    # a = f"({i}, {row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]}, {row[5]})"
    query = f"""
          INSERT INTO {table} {cols}
          VALUES
          {b}
          """
    # print (query)
    cursor.execute(query)
    conn.commit()
    i += 1
  # print (query)

In [35]:
## implement the top-highest-franchise-anime-series
cols0 = '(series_id, title, date_release, status, runtime_per_episode, episode_count, others)'
insertion('top_series', series_table, cols0, cur)

* After using inserting function to take the values into the table, we will have some queries to request some essential data for analysis process.

In [36]:
## select some values from 'top_series' table
qry = """
      SELECT * FROM top_series WHERE status = 'Ongoing'
      """
b = cur.execute(qry)
res = b.fetchall()
print (res)

qry = """
      SELECT COUNT(status)
      FROM top_series
      WHERE status = 'Ongoing';
      """
c = cur.execute(qry)
res1 = c.fetchall()
print (res1)

qry = """
      SELECT COUNT(status)
      FROM top_series
      WHERE status = 'Ongoing' AND runtime_per_episode = '22-24 minutes' OR runtime_per_episode = '24 minutes';
      """
d = cur.execute(qry)
res2 = d.fetchall()
print (res2)

qry = """
      SELECT *, (episode_count + others) AS total_count
      FROM top_series
      """
total_count = cur.execute(qry)
res3 = total_count.fetchall()
print (res3)

qry = """
      SELECT episode_count, others
      FROM top_series
      """
ratio_episode_count_others = cur.execute(qry)
res4 = ratio_episode_count_others.fetchall()
print (res4)

qry = """
      SELECT (episode_count / others)
      FROM top_series
      WHERE others > 0 AND episode_count > 0
      """
ratio_episode_count_others = cur.execute(qry)
res5 = ratio_episode_count_others.fetchall()
print (res5)

qry = """
      SELECT *
      FROM top_series
      WHERE (episode_count / others) == 2
      """
ratio_episode_count_others = cur.execute(qry)
res6 = ratio_episode_count_others.fetchall()
print (res6)

[]
[(0,)]
[(0,)]
[(1, 'Doraemon', '1973', '2022', '22', 3003, 67, 3070), (2, 'Sazae-san', '1969', '2022', '22', 2640, 0, 2640), (3, 'Nintama Rantarō', '1993', '2022', '10', 2199, 3, 2202), (4, 'Ojarumaru', '1998', '2022', '10', 1827, 4, 1831), (5, 'Oyako Club', '1994', '2013', '5', 1818, 0, 1818), (6, 'Soreike! Anpanman', '1988', '2022', '24', 1587, 113, 1700), (7, 'Kirin Monoshiri Yakata', '1975', '1979', '5', 1565, 0, 1565), (8, 'Chibi Maruko-chan', '1990', '2022', '24', 1493, 64, 1557), (9, 'Kirin Ashita no Calendar', '1980', '1984', '5', 1498, 0, 1498), (10, 'Manga Nippon Mukashi Banashi', '1975', '1995', '24', 1494, 0, 1494), (11, 'Hoka Hoka Kazoku', '1976', '1982', '5', 1428, 0, 1428), (12, 'Shimajirō', '1993', '2022', '24', 1403, 8, 1411), (13, 'Pokémon', '1997', '2022', '24', 1220, 144, 1364), (14, 'Monoshiri Daigaku Ashita no Calendar', '1966', '1970', '5', 1274, 0, 1274), (15, 'Transformers', '1984', '2022', '0', 1199, 5, 1204), (16, 'Crayon Shin-chan', '1992', '2022', '24', 

* We can see that from getting data about the ratio of some films' episode counts and others, there is a fact that these two have no relationship. The three films having the lowest ratios (which is nearly equal to 2) are Naruto, Gundam, and Dragon Ball. We also do same thing with 'top_series' table.

In [37]:
## implement the top-highest-grossing-anime-movies
cols1 = '(movie_id, title, revenue, year_release, genre, others)'
insertion('top_movies', movies_japan_revenue_table, cols1, cur)

In [38]:
qry = """
      SELECT COUNT(*) FROM top_movies WHERE revenue > 100000000 OR revenue == 100000000
      """
revenue_higher_1m = cur.execute(qry)
res7 = revenue_higher_1m.fetchall()
print (res7)

[(22,)]


In [39]:
qry = f'''
        ALTER TABLE top_movies
        ADD revenue_category TEXT(50);
        '''
cur.execute(qry)

* Also, we want to modify the 'top_movies' by categorizing the film's revenue. We use 'UPDATE' statement to add new column to existing table and insert new value into it.

In [40]:
n = len(movies_japan_revenue_table)

# categorize the films based on revenues with 'UPDATE' statement
for i in range (n):
  id = i + 1
  if movies_japan_revenue_table[i][1] >= 500000000:
    qry = f"""
          UPDATE top_movies SET revenue_category = '500M+' WHERE movie_id = {id};
          """
  elif movies_japan_revenue_table[i][1] >= 100000000 and movies_japan_revenue_table[i][1] < 500000000:
    qry = f"""
          UPDATE top_movies SET revenue_category = 'Between 100M and 500M' WHERE movie_id = {id};
          """
  elif movies_japan_revenue_table[i][1] < 100000000 and movies_japan_revenue_table[i][1] >= 50000000:
    qry = f"""
          UPDATE top_movies SET revenue_category = 'Between 50M and 100M' WHERE movie_id = {id};
          """
  else:
    qry = f"""
          UPDATE top_movies SET revenue_category = '<50M' WHERE movie_id = {id};
          """
  cur.execute(qry)
  conn.commit()


In [41]:
qry = '''
      SELECT * FROM top_movies
      '''
all = cur.execute(qry)
res8 = all.fetchall()
print (res8)

qry = '''
      SELECT COUNT(*) FROM top_movies
      WHERE revenue_category == 'Between 100M and 500M'
      '''
revenue_100_500 = cur.execute(qry)
res9 = revenue_100_500.fetchall()
print (res9)

qry = '''
      SELECT COUNT(*) FROM top_movies
      WHERE revenue_category == 'Between 50M and 100M'
      '''
revenue_below_50 = cur.execute(qry)
res10 = revenue_below_50.fetchall()
print (res10)


[(1, 'Demon Slayer the Movie: Mugen Train', 506523013, '2020', 'Anime', 0, '500M+'), (2, 'Spirited Away', 395580000, '2001', 'Anime', 0, 'Between 100M and 500M'), (3, 'Your Name', 380140500, '2016', 'Anime', 0, 'Between 100M and 500M'), (4, "Howl's Moving Castle", 236323601, '2004', 'Anime', 0, 'Between 100M and 500M'), (5, 'One Piece Film: Red', 206740295, '2022', 'Anime', 0, 'Between 100M and 500M'), (6, 'Ponyo', 204826668, '2008', 'Anime', 0, 'Between 100M and 500M'), (7, 'Jujutsu Kaisen 0', 195870885, '2021', 'Anime', 0, 'Between 100M and 500M'), (8, 'Weathering with You', 193715360, '2019', 'Anime', 0, 'Between 100M and 500M'), (9, 'Stand by Me Doraemon', 183442714, '2014', 'Anime', 0, 'Between 100M and 500M'), (10, 'Pokémon: The First Movie', 172744662, '1998', 'Anime', 0, 'Between 100M and 500M'), (11, 'Princess Mononoke', 170005875, '1997', 'Anime', 0, 'Between 100M and 500M'), (12, 'Bayside Shakedown 2', 164450000, '2003', '0', 0, 'Between 100M and 500M'), (13, 'The Secret Wor

* We can see that all of the films having revenue in two main groups: 'Between 100M and 500M' and 'Between 50M and 100M'. Only one film 'Demon Slayer The Movie' has the revenue beyond 500M (USD).

In [42]:
### end SQL DATABASE

#### Storage Challenges (requirement 6):

When working on the insertion() function, we had several fail attempts to make the SQL tables because we didn't realize that the data values were a mix of string and numeric. We had to tuple them and turn them into string.

## DATA ANALYSIS AND VISUALIZATION

(requirement 6)
* Our first created table is top_series, implementing the top highest franchise anime series. We select the following columns to create the table: series_id, title, date_release, status, runtime_per_episode, episode_count, others. 

* Our second created table is top_movies, the top highest grossing anime movies. We select the following columns to create the table: movie_id, title, revenue, year_release, genre, others. Then we update a new column, or variable, called revenue_category. This new variable has the following values: 500M+, Between 100M and 500M, Between 50M and 100M. 




*Markdown cells explaining how you use the data frame(s) to process the data. (Requirement #6).*

*A final Markdown cell explaining the answer(s) to your central question (Requirement #7).*

In [43]:
import plotly.express as px
import matplotlib.pyplot as plt
# Top 50 Highest Grossing Anime Worldwide
# Scatter plot of Revenue (USD) over Year release for each anime movie, with the revenue also indicated by the bubble size
fig1 = px.scatter(df0, x="Year release", y="Revenue (USD)", size="Revenue (USD)", color="Movie name", log_x=1000, size_max=40)
fig1.update_layout(
    title={'text': "Movies' Revenues (USD) over Year release (Worldwide)"})

fig1.show()

We use df0 - Top 50 Highest Grossing Anime Worldwide - to plot a scatter chart for the year release and worldwide revenue of top 50 animes. Year release is our x-axis. Worldwide revenue in USD is our y-axis and also is indicated by the width of each data point's bubble. Each anime name is differentiated by colors. 

As we observe for the scatterplot above, we can see determine the top 5 movies with the highest revenue: “Demon Slayer the Movie: Mugen Train”, “Spirited Way”, “Your Name,” “Howl’s Moving Castle,” and “One Piece Film: Red.” for the Worldwide market. Look at each point, we can also view the attached information, such as Year Release and Revenue. 

In [44]:
# Top 50 Highest Grossing Anime In Japan
# Scatter plot of Revenue (Yen) over Year release for each anime movie, with the revenue also indicated by the bubble size
fig2 = px.scatter(df1, x="Year release", y="Revenue (Yen)", size="Revenue (Yen)", color="Movie name", log_x=1000, size_max=40)
fig2.update_layout(
    title = {
        'text': "Movies' Revenues (Yen) over Year release (Japan)"})
fig2.show()

We use df1 - Top 50 Highest Grossing Anime In Japan - to plot a scatter chart for the year release and revenue of top 50 animes in Japanese market. Year release is our x-axis. Japan revenue in Yen is our y-axis and also is indicated by the width of each data point's bubble. Each anime name is differentiated by colors. 

With a similar scatterplot, we can see the top 5 movies with the highest revenue in Japan. Interestingly, these top 5 are closely the same as the worldwide ones. The only difference is the 5th ranking, for Japan, it is “Princess Mononoke,” and for the worldwide, it is “One Piece Film.” Comparing these two graphs, we can conclude that the number of anime movies in worldwide is more trending than in Japan.

In [45]:
# Top 50 Highest Grossing Anime Worldwide
# Scatter plot of Revenue (USD) over Year release for each anime movie, based on the two main genres (Anime and Live-action) with the revenue also indicated by the bubble size
fig3 = px.scatter(df0, x="Year release", y="Revenue (USD)", size="Revenue (USD)", color="Genre", log_x=1000, size_max=40)
fig3.update_layout(
    title = {
        'text': 'Movies Revenues (USD) over Year release (Worldwide), categorized by Genre'})
fig3.show()

We use df0 - Top 50 Highest Grossing Anime Worldwide - to plot a scatter chart for the year release, worldwide revenue, and genres of top 50 animes. Year release is our x-axis. Worldwide revenue in USD is our y-axis and also is indicated by the width of each data point's bubble. Each anime genre is differentiated by colors. Genre 0 is the unknown genre. 

Instead of analyzing the movie under the revenue category, in this scatterplot, we want to observe the movie genre factors. There are two main movie genres: anime and live-action. It is clear in the graph that the number of anime movies is much more than the number of live-action, which also leads to more profits. The evidence is top 10 anime movie with the highest revenue is all anime genre. 

In [46]:
# Top 50 Highest Grossing Anime In Japan
# Scatter plot of Revenue (Yen) over Year release for each anime movie, based on the two main genres (Anime and Live-action) with the revenue also indicated by the bubble size
fig4 = px.scatter(df1, x="Year release", y="Revenue (Yen)", size="Revenue (Yen)", color="Genre", log_x=1000, size_max=40)
fig4.update_layout(
    title = {
        'text': 'Movies Revenues (Yen) over Year release (Japan), categorized by Genre'})
fig4.show()

We use df1 - Top 50 Highest Grossing Anime In Japan - to plot a scatter chart for the year release, Japan's revenue, and genres of top 50 animes. Year release is our x-axis. Japan's revenue in Yen is our y-axis and also is indicated by the width of each data point's bubble. Each anime genre is differentiated by colors. 

In Japan, the number of anime movies also exceeds the number of live-action movies. Especially, the period from 2015 to 2020 remark a period that the anime movie makes huge profits than the live-action. The best movie, “Demon Slayer the Movie: Mugen Train,” with 40,430,000,000 yen revenue also is made in this period (2020). Comparing these two movie-genre scatterplot, we can take an insight that the audience taste in worldwide and Japanese is kind of similar - however, the movie sold in worldwide makes more profit than in Japan. 

In [47]:
# 50 Anime series' franchises by episode count

# Scatter plot of Anime series' Runtimes (min) over Year release, with their linear fit trendline
fig5 = px.scatter(df, x="Year release", y="Runtime (min on TV)", trendline="ols", log_x=100, size_max=40)
fig5.update_layout(title = {'text': 'Runtime (min) of Anime Series over Year release'})
fig5.show()

We used df - 50 Anime series' franchises by episode count - to plot a scatter chart for the year release and runtime in min of a sample of 50 anime series as well as their fit line. Year release is set as x-axis. Runtime is set as y-axis. The fit line is linear fit line that also shows R^2 that indicates how well the model fits the observed data. 

The scatter plot shows a positive relationship between the runtime (min) of anime series over their year release, as indicated by the upward linear trendline. The runtime for each episode of the anime series gets longer over years of release. However, the R^2 is low -10.7%, indicating that the regression model doesn't fit the observed data well.

There are a lot of outliers to this fit line. A lot of anime that started long ago have the same amount of runtime as the newer animes



In [48]:
# Scatter plot of Anime series' Runtimes (min) over End Broadcasting Year, with their linear fit trendline
fig6 = px.scatter(df, x="End Broadcasting Year", y="Runtime (min on TV)", trendline="ols", color_discrete_sequence=['crimson'], log_x=100, size_max=40)
fig6.update_layout(title = {'text': "Runtime (min) of Anime Series over End Broadcasting Year"})
fig6.show()

We used df - 50 Anime series by franchise series total episode count - to plot a scatter chart for the ending year and runtime in min of a sample of 50 anime series as well as their fit line. Ending year is set as x-axis. Runtime is set as y-axis. The fit line is linear fit line that also shows R^2 that indicates how well the model fits the observed data. 

The scatter plot shows a positive relationship between the runtime (min) of anime series and their ending year, as indicated by the upward linear trendline. The runtime for each episode of the anime series gets longer over the end broadcasting years. However, the R^2 is moderately low - 37.8%, indicating that the regression model doesn't fit the observed data well.

There are fewer outliers compared to the relationship of runtime with the year of release. The r^2 is also higher. This implies that runtime might be more strongly associated with the ending year


In [49]:
# Scatter plot of Anime series' Total counts of eps over Runtimes (min), with the counts of Others(filler eps)
fig7 = px.scatter(df_v1, x="Runtime (min on TV)", y="Total count", size='Others', color='Series name', log_x=100, size_max=40)
fig7.update_layout(title = {'text': 'Episode count of Anime Series over Runtime (min)'})
fig7.show()

We used df_v1 - 50 Anime series by franchise series total episode count, except the Episodes count - to plot a scatter chart for runtime in min, total count, and Others count of a sample of 50 series. Runtime in min is our x-axis. Total count is our y-axis. Others (filler eps' count) is our bubble size. The series' names are differentiated by colors. 


The series with highest runtime (22-24 minutes) have the highest Others count (filler eps or OVA), and also higher Total counts of eps, between 500 and 2000. Doraemon is an exception that has low Others' count but an outstandingly high Total count. 

In [50]:
japan_overseas = {'Japan': 0, 'Overseas': 0}
for x in table2:
  if type(x[2]) is not str and type(x[3]) is not str:
    japan_overseas['Japan'] += x[2]
    japan_overseas['Overseas'] += x[3]
print (japan_overseas)


{'Japan': 317610910, 'Overseas': 835507821}


In [51]:
# Top 50 box ticket sales all the time
# Ticket sales in Japan and Overseas
import plotly.graph_objects as go

fig8 = go.Figure(data=[go.Pie(labels=list(japan_overseas.keys()), values=list(japan_overseas.values()))])
fig8.update_layout(title = {'text': 'Ticket Sales of Top 50 in Japan and Overseas'})
fig8.show()

We used df2 - Top 50 box ticket sales all the time - to plot a pie chart for the total ticket sales of the top 50 in Japan and Overseas. We computed the sum of ticket sales for the two regional categories. The categories are separated by colors and part of the pie. The whole pie can be seen as worldwide. 

For the pie chart, we want to compare the ticket sale between Japan and oversea. As we see clearly in the graph, in Japan, ticket sale is 27.5%, while oversea, it is 72.5%. Therefore, the profit of an anime movie is taken mostly from the total of other countries outside Japan. 

In [52]:
def convert(s):
  if '–' in s: 
    und = s.find('–')
    return s[0:und-1], s[und+2:]
  else: 
    return '-1', '-1'
# print (convert('South Korea (Seoul City) – 20,280'))

table3[0] = table3[0][0:2] + ['China – 400,000,000']
table3.pop(1)
table3[0] = table3[0]
ctr_ads = {'China': 0, 'South Korea': 0, 'United States and Canada': 0, 'Europe':0}
eur = ['France and Germany', 'France', 'Spain', 'Other EU territories (since 1996)', 'Russia', 'Netherlands']
for x in table3:
  a = x[1:]
  # print (x[0])
  for y in a:
    if y != None: 
      country, tickets = convert(y)
      # print (country, tickets)
      if country == 'South Korea (Seoul City)': country = 'South Korea'
      if country == 'United States': country = 'United States and Canada'
      if country in ctr_ads.keys():
        if tickets != '3.02':
          tickets = tickets.replace(',', '')
          tickets = tickets.replace('+', '')
          tickets = int(tickets.replace('\n', ''))
          ctr_ads[country] += tickets
        else:
          ctr_ads['South Korea'] += 3020000
      else:
        if country in eur or 'Europe' in country:
          # print (country)
          tickets = tickets.replace(',', '')
          tickets = tickets.replace('+', '')
          tickets = int(tickets.replace('\n', ''))
          ctr_ads['Europe'] += tickets
print (ctr_ads)

{'China': 502115125, 'South Korea': 14971454, 'United States and Canada': 52996198, 'Europe': 40798754}


In [53]:
import plotly.graph_objects as go

# Ticket sales admission in some continents/countries

fig9 = go.Figure(data=[go.Pie(labels=list(ctr_ads.keys()), values=list(ctr_ads.values()))])
fig9.update_layout(title = {'text': 'Ticket sales admission in some major markets'})
fig9.show()

We used the new data found in the url for 50 Anime series' franchises by episode count to calculate the total ticket sales in 4 different major markets - China, US and Canada, Europe, and South Korea. Then we created a pie chart to compare the success of those 4 markets. This sample of anime is smaller, using only animes that are published in those markets and have sales' reports. 

In this graph, we consider some countries oversea which consume the largest ticket sale due to the following: China (82.4%), United States - Canada (8.65%), Europe (6.41%), and South Korea (2.54%). This pie chart will help us to take a deep look at the ticket sale and revenue of anime movie in oversea in general. 

## Discussion and Conclusion
* Considering top 50 animes, the newer ones have the higher revenue (USD) worldwide and in Japan. 
* Anime genre produces higher revenues than the live-action genre and other unknown genres both worldwide and in Japan. Anime genre is also the more popular genre recently than the live-action, which is becoming less popular due to the fact that its profit remains stably low over time. Meanwhile, the anime genre has more profitable series/ movies. 
* From the sample of 50 series, anime series' runtime has a weak positive relation to the year release and a moderate positive relation to the ending year. Thus, newer anime and anime that ends earlier have a higher runtime per episode.
* Anime series that have the most side or filler episodes (OVAs) usually have total counts of episodes between 500 and 2000, and usually have the highest runtime/ep (about 22-24 minutes). In other words, anime series with the highest runtime usually have more filler episodes and OVAs and total counts between 500 and 2000. Doraemon is an exception in the sample of 50 series, having fewer filler eps but the highest and most outstanding total count of eps. This implies that it is a very quality and profitable anime series. 
* In Japan, box office ticket sale is 27.5%, while overseas, it is 72.5%. Anime has been entering the global markets for a long time now, so there have been changes in their plots and characters to adopt different cultures and ideologies. Japan's market still plays a great role in anime's profit. 
* Between China, US and Canada, Europe, and South Korea, China's market has the highest and most overwelming ticket sales even though the country bans some animes. The market in US and Canada is the next most profittable among them. 